In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys
import datetime
from datetime import datetime
from astropy.coordinates import SkyCoord

In [2]:
sys.path.insert(0, os.path.abspath('beam-model'))
from beam_model import utils, formed

In [7]:
sys.path.append("/arc/home/mseth/.local/lib/python3.11/site-packages")
import astrodendro

In [8]:
sys.path.insert(0, '/arc/home/mseth/frb-calibration-master')
from frb_calibration import intensity_calibration_helpers as ich

ModuleNotFoundError: No module named 'statsmodels'

Stuff we define beforehand

In [ ]:
freqs = np.linspace(400.390625, 800, 1024)            # The data has 256 frequency channels
has = np.array(np.linspace(-105, 104.90278, 1024))   # 1024 HAs

source_name = "CYG_A"
coords = SkyCoord.from_name(source_name)
source_ra = coords.ra.deg
source_dec = coords.dec.deg


# BF to JY conversion equation
def bf_to_jy(bf_spectrum, f_good):
    factor = (np.square(1024) * 128) / (np.square(4) * 0.806745 * 400)
    result = bf_spectrum / ( factor * np.square(f_good) ) 
    return result

Data

In [ ]:
## DATA ##
# Load in data files

files = np.load('/arc/projects/chime_frb/mseth/cyg_A/frb_CYG_A_2025-07-11_beam_1105.npz', allow_pickle=True)

In [ ]:
spectra = files['spectra']
ts = np.mean(spectra, axis=0)

max_tidx = np.argmax(ts)
max_timestamp = ich.get_calibrator_transit(source_ra, source_dec, "20250711")
spectra_at_peak = spectra[:, max_tidx]
beam_id = 1105 

ha, y_at_peak = utils.get_position_from_equatorial(source_ra, source_dec, max_timestamp)

In [ ]:
plt.figure()
plt.plot(freqs, spectra_at_peak)
plt.title("Example of uncorrected data")
plt.show()

Beams & Correcting

In [ ]:
## PRIMARY BEAM RESPONSE ##

# Load in holography data 
path = "/users/mseth/holography_data/npz_files"

npz_files = []
for (root, dirs, file) in os.walk(path):
    for f in file: 
        npz_files.append(os.path.join(root, f))
xx_list = []
yy_list = []
for file in npz_files:
    data = np.load(file)
    xx = data['XX']
    yy = data['YY']
    xx_list.append(xx)
    yy_list.append(yy)

xx = np.vstack(xx_list)
yy = np.vstack(yy_list)
intensity = xx + yy / 2

# Get spectra at every peak HA 
presponse_norm = []

for ha in has_at_peak:
    idx = (np.abs(has - ha)).argmin()         # Find index of closest HA to HA at peak
    xx_response = xx[:,idx]
    yy_response = yy[:, idx]
    
    int_response = intensity[:,idx]           # Get intensity spectrum at that HA 
    int_masked = intensity[:,1760:1840]       # Get maximum intensity in the main beam
    inty_max = np.max(intensity_masked)       
    
    int_norm = int_response / intensity_max   # Normalize intensity spectrum at the HA
    presponse_norm.append(int_norm)

In [ ]:
plt.figure()
plt.plot(np.linspace(400.390625, 800, 1024), np.log10(int_response[0]))
plt.title("Example of primary beam response")
plt.show()

In [ ]:
## CORRECTING FOR PRIMARY BEAM RESPONSE ##

p_corrected_sp = []

for i, (spectra, presponse) in enumerate(zip(spectra_at_peak, presponse_norm)):
    presponse_ds = int_response[::4]                             # Downsample primary response spectrum from 1024 freq channels to 256 (matching data)
    spectra_at_peak_corrected = spectra_at_peak / presponse_ds   # Correction
    p_corrected_sp.append(spectra_at_peak_corrected)

In [ ]:
## SYNTHESIZED BEAM RESPONSE + CORRECTING

sbm = formed.FFTFormedBeamModel()

calibrated_spectra = []

for i, (beam_id, ha, y, spectrum) in enumerate(zip(beam_ids, has_at_peak, y_at_peak, p_corrected_sp)):
    sensitivity = sbm.get_sensitivity([beam_id], np.array([ha, y_at_peak]).T, freqs).squeeze()
    
    mean_sensitivity = np.nanmean(sensitivity, axis=1)            # Average sensitivity across frequencies
    peak_sensitivity_ind = np.argmax(mean_sensitivity)            
    sensitivity_at_peak = sensitivity[peak_sensitivity_ind, :]    # Spectrum of synthesized response at its peak

    spectra_corrected = spectrum / sensitivity_at_peak            # Correction
    spectra_jy = bf_to_jy(spectra_corrected, 1)                   # Conversion
    calibrated_spectra.append(spectra_jy)